In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model

In [2]:
train=[]
train_y=[]

for i in range(1,5001):
    train.append(pd.read_csv('./masked_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    train_y.append(pd.read_csv('./dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())

train = np.array(train)
train_y = np.array(train_y)

In [3]:
train = train.swapaxes(1,2)
train_y = train_y.swapaxes(1,2)

In [4]:
model = tf.keras.models.Sequential()

In [5]:
# model.add(Input(shape=(9,32)))

# inputs = Input(shape=(9,32))
# lstm =  tf.keras.layers.Bidirectional(LSTM(units=32, return_sequences=True, activation='sigmoid'))(inputs)
# outputs = tf.keras.layers.Bidirectional(LSTM(units=32, return_sequences=False, activation='sigmoid'))(lstm)
# forward = LSTM(units=16, return_sequences=True, activation='sigmoid')
# backward = LSTM(units=16, return_sequences=True, activation='sigmoid', go_backwards=True)

# model.add(tf.keras.layers.Bidirectional(forward, backward_layer=backward, input_shape=(9,32)))
# model.add(LSTM(units=32, return_sequences=True, activation='sigmoid', input_shape=(9,32)))
model.add(LSTM(units=9, return_sequences=True, activation='sigmoid', input_shape=(32,9)))
# model = Model(inputs, outputs)

In [6]:
# model.compile(loss = tf.keras.losses.binary_crossentropy, optimizer='rmsprop')
model.compile(loss = tf.keras.losses.mse, optimizer='adam')

In [ ]:
model.fit(train.astype('float32'),train_y.astype('float32'), epochs=40)

In [7]:
test_x=[]
test_y=[]

for i in range(10000,10100):
    test_x.append(pd.read_csv('./masked_dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    test_y.append(pd.read_csv('./dataset/beat/'+str(i)+'.csv.gzip', compression='gzip').to_numpy())
    
test_x = np.array(test_x)
test_y = np.array(test_y) 

In [8]:
test_x = test_x.swapaxes(1,2)
test_y = test_y.swapaxes(1,2)

In [10]:
dec = np.zeros_like(train)

for i in range(dec.shape[0]):
    for j in range(dec.shape[1]):
        if j != 0:
            dec[i][j] = train_y[i][j-1]

In [11]:
decn = np.zeros_like(test_y)
for i in range(decn.shape[0]):
    for j in range(decn.shape[1]):
        if j != 0:
            decn[i][j] = test_y[i][j-1]

In [ ]:
asdf = model.predict(test_x)

asdf = asdf.round()

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
i = 10
print(test_x[i])
print('-'*10)
print(asdf[i].astype(int))
print('-'*10)
print(test_y[i])

--

In [12]:
encoder_inputs = Input(shape=(32,9))
encoder_lstm = LSTM(units=512, return_state=True, dropout=0.1, recurrent_dropout=0.1)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [13]:
decoder_inputs = Input(shape=(None,9))
decoder_lstm = LSTM(units=512, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.1)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(9, activation='sigmoid')

decoder_outputs = decoder_dense(decoder_outputs)


In [14]:
# if model is not None:
del model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE)

In [17]:
with tf.device('/gpu:0'):
    model.fit(x=[train, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.2)

Epoch 1/5000
125/125 [==============================] - 31s 228ms/step - loss: 0.1013 - val_loss: 0.0951
Epoch 2/5000
125/125 [==============================] - 28s 223ms/step - loss: 0.0856 - val_loss: 0.0841
Epoch 3/5000
125/125 [==============================] - 28s 224ms/step - loss: 0.0774 - val_loss: 0.0793
Epoch 4/5000
125/125 [==============================] - 27s 219ms/step - loss: 0.0732 - val_loss: 0.0762
Epoch 5/5000
125/125 [==============================] - 28s 225ms/step - loss: 0.0705 - val_loss: 0.0738
Epoch 6/5000
125/125 [==============================] - 28s 220ms/step - loss: 0.0670 - val_loss: 0.0706
Epoch 7/5000
125/125 [==============================] - 28s 228ms/step - loss: 0.0639 - val_loss: 0.0665
Epoch 8/5000
125/125 [==============================] - 28s 223ms/step - loss: 0.0600 - val_loss: 0.0638
Epoch 9/5000
125/125 [==============================] - 27s 220ms/step - loss: 0.0571 - val_loss: 0.0625
Epoch 10/5000
125/125 [==============================] 

In [78]:
model = tf.keras.models.load_model('LSTM_EnDe_Flip_testloss02806.h5')

In [18]:
model.evaluate(x=[test_x, decn], y=test_y)

4/4 [==============================] - 0s 74ms/step - loss: 0.0588


0.05883701145648956

In [19]:
asd = model.predict(x=[test_x, decn])
asd = asd.round()

In [20]:
i = 10
print(test_x[i])
print('-'*10)
print(asd[i].astype(int))
print('-'*10)
print(test_y[i])

[[0 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]]
----------
[[0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 

In [61]:
model.save('LSTM_EnDe_testloss3225.h5')

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)

In [22]:
decoder_state_input_h = Input(shape=(512,))
decoder_state_input_c = Input(shape=(512,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [23]:
n_features = 9
n_timesteps_in = 32

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, n_features))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = list()
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # convert the token/output prediction to a token/output
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_digit = sampled_token_index
        # add the predicted token/output to output sequence
        decoded_seq.append(sampled_digit)
        

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_digit == '\n' or
           len(decoded_seq) == n_timesteps_in):
            stop_condition = True

        # Update the input target sequence (of length 1) 
        # with the predicted token/output 
        target_seq = np.zeros((1, 1, n_features))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update input states (context vector) 
        # with the ouputed states
        states_value = [h, c]

        # loop back.....
        
    # when loop exists return the output sequence
    return decoded_seq